In [ ]:
!nvidia-smi

In [ ]:
# --- CLEAN START ---
# Remove everything that can conflict
# !pip uninstall -y unsloth unsloth-zoo transformers accelerate peft datasets tokenizers huggingface-hub bitsandbytes sentencepiece

# --- PYTORCH (CUDA 11.8) ---
# !pip install torch --index-url https://download.pytorch.org/whl/cu118

# --- INSTALL MODERN UNSLOTH (the correct one for Qwen2.5) ---
# !pip install "unsloth==2026.2.1"

# --- INSTALL THE HF STACK COMPATIBLE WITH UNSLOTH 2026.x ---
# !pip install "transformers>=4.57.0" "accelerate>=0.34.1" "peft>=0.18.0" datasets sentencepiece bitsandbytes

# Clean uninstall - remove everything that could conflict
!pip uninstall -y unsloth unsloth-zoo transformers accelerate peft datasets tokenizers huggingface-hub bitsandbytes sentencepiece trl xformers

# Install pinned PyTorch for CUDA 11.8 (good for T4)
!pip install torch --index-url https://download.pytorch.org/whl/cu118

# Install exact Unsloth 2026.2.1 release
!pip install "unsloth==2026.2.1"

# Force-reinstall unsloth_zoo right after to match the 2026.2.1 expectations
!pip install --no-cache-dir --force-reinstall "unsloth_zoo @ git+https://github.com/unslothai/unsloth-zoo.git@February-2026"

# Compatible HF ecosystem versions (your original bounds are fine, but explicit helps)
!pip install "transformers>=4.57.0,<4.58.0" "accelerate>=0.34.1" "peft>=0.18.0" "datasets" "sentencepiece" "bitsandbytes" "trl<0.25.0"

##########################################
# 1. Install Unsloth with the latest Colab-specific optimization
# !pip install --force-reinstall --no-cache-dir --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --no-cache-dir unsloth_zoo
# Force-update both packages together to stay in sync
# !pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --upgrade --no-cache-dir unsloth_zoo
# !pip uninstall -y unsloth unsloth-zoo
# !pip install --upgrade --no-cache-dir "unsloth[colab-new]@git+https://github.com/unslothai/unsloth.git"
# !pip install --upgrade --no-cache-dir unsloth_zoo

# 2. Install essential backends for LoRA and 4-bit quantization
#!pip install --no-deps "xformers<0.0.30" "trl<0.13.0" peft accelerate bitsandbytes

# 3. Install data handling and high-speed download utilities
#!pip install datasets sentencepiece hf_transfer huggingface_hub

# 4. Enable HF_TRANSFER for 10x faster model loading into Colab's local disk
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

In [ ]:
import unsloth; print(unsloth.__version__)
import unsloth_zoo; print(unsloth_zoo.__version__ if hasattr(unsloth_zoo, '__version__') else "no __version__ attr")
!pip show unsloth unsloth-zoo | grep -E "Name|Version|Location"

In [ ]:
from google.colab import drive
import torch
import os
drive.mount('/content/drive')

with open('/content/drive/MyDrive/TMP/HF_TOKEN.txt') as f:
    token = f.read().strip()
    if token:
        os.environ["HF_TOKEN"] = token
# 2. Check if HF_TOKEN is set in the environment
test_token=os.getenv("HF_TOKEN")
if test_token:
    print("HF_TOKEN is set in the environment.")
else:
    print("HF_TOKEN is NOT set in the environment. Please check your token file and try again.")
    raise ValueError("HF_TOKEN not found in environment variables.")

# 2. Detect Hardware for optimization
major_v, _ = torch.cuda.get_device_capability()
IS_A100 = True if major_v >= 8 else False
print(f"🚀 Hardware: {'A100 (using BF16)' if IS_A100 else 'T4 (using FP16)'}")

In [ ]:
import os

paths = {
    "Script": "/content/drive/MyDrive/scripts/train_scout_unsloth.py",
    "Data": "/content/drive/MyDrive/data/train/detective_finetune.jsonl"
}

for name, path in paths.items():
    if os.path.exists(path):
        print(f"✅ {name} found: {path}")
    else:
        print(f"❌ {name} MISSING: {path}")
        raise FileNotFoundError(f"{name} not found at {path}. Please check the path and try again.")

In [ ]:
import time
import os
# Force memory management fixes before running the script
os.environ["PYTORCH_ALLOC_CONF"] = "expandable_segments:True"
# Force torch to avoid using the broken dynamo compiler
os.environ["TORCHDYNAMO_DISABLE"] = "1"

script_path = "/content/drive/MyDrive/scripts/train_scout_unsloth.py"

print("🎬 Initializing fine-tuning session...")
start_time = time.time()

# Run the training script
!python {script_path}

end_time = time.time()
elapsed_mins = (end_time - start_time) / 60

print(f"\n" + "="*40)
print(f"🏁 TRAINING SESSION COMPLETE")
print(f"⏱️ Total Time: {elapsed_mins:.2f} minutes")
print(f"📂 Output Location: /content/drive/MyDrive/outputs/detective-qwen-sft")
print("="*40)